# MESFET

## Metal-Semiconductor Field-Effect Transistor

The MESFET uses a Schottky (metal-semiconductor) junction as its gate, making it simpler in fabrication than a MOSFET while still offering voltage-controlled current modulation. MESFETs are widely used in high-frequency applications due to their thin active layer and high electron mobility.

**Learning Objectives:**
- Understand MESFET structure and Schottky-gate operation
- Simulate output characteristics (Id vs Vds) with PADRE
- Simulate transfer characteristics (Id vs Vgs) with PADRE
- Analyze the effect of gate workfunction and channel doping
- Extract threshold voltage and transconductance

In [ ]:
# Setup: Load PADRE environment (required on nanoHUB)
# This cell loads the PADRE simulator into your environment.
# If running locally with PADRE already in your PATH, this will be skipped gracefully.

from nanohubpadre import use

# Load the PADRE simulator environment
%use padre-2.4E-r15

print("PADRE environment setup complete.")

---

## Device Parameters Reference

The `describe()` function shows all available parameters for the MESFET factory, including geometry, doping, physical models, and sweep options.

In [ ]:
from nanohubpadre import Simulation

# Show all available parameters for the MESFET
Simulation.describe('mesfet')

---

## 1. MESFET Physics

### 1.1 Structure

```
     Gate Metal
     ==========          ← Schottky contact (electrode 3)
  Source  Channel  Drain
  (N+)   (N-type)  (N+)  ← Active layer
  ====   --------  ====
    |                |
    |   P-Substrate  |    ← Isolation layer
    +----------------+
```

The key difference from a MOSFET is the absence of a gate oxide. The metal gate forms a **Schottky barrier** directly with the semiconductor channel. Applying a negative gate voltage widens the depletion region under the gate, pinching the channel and reducing drain current.

### 1.2 Key Parameters

- **Schottky barrier height**:
  $$\phi_B = \phi_m - \chi$$
  where $\phi_m$ is the gate metal workfunction and $\chi = 4.05\,\text{eV}$ is the silicon electron affinity.

- **Built-in potential**:
  $$V_{bi} = \phi_B - \frac{kT}{q}\ln\left(\frac{N_c}{N_d}\right)$$

- **Maximum depletion width** (full pinch-off):
  $$W_{\max} = \sqrt{\frac{2\,\varepsilon_s\,V_{bi}}{q\,N_d}}$$

- **Threshold (pinch-off) voltage**:
  $$V_P = -\frac{q\,N_d\,a^2}{2\,\varepsilon_s}$$
  where $a$ is the channel depth. The device turns off when $V_{gs} \approx V_P - V_{bi}$.

- **Drain current** (gradual-channel approximation, linear region):
  $$I_d = \frac{q\,\mu_n\,N_d\,W\,a}{L}\left[V_{ds} - \frac{1}{3}\left((V_{bi}-V_{gs}-V_{ds})^{3/2} - (V_{bi}-V_{gs})^{3/2}\right)\cdot\frac{1}{\sqrt{V_{bi}-V_{gs}}}\right]$$

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from nanohubpadre import create_mesfet

# Physical constants
q        = 1.6e-19       # Elementary charge (C)
eps_si   = 11.7 * 8.85e-14  # Silicon permittivity (F/cm)
kT       = 0.0259        # Thermal voltage at 300 K (eV)
chi_si   = 4.05          # Silicon electron affinity (eV)
Nc       = 2.8e19        # Effective density of states in CB (cm⁻³)

# Default MESFET geometry (matches create_mesfet defaults, in µm)
GATE_LENGTH    = 0.2     # Gate length
CHANNEL_LENGTH = 0.2     # Source-to-gate / gate-to-drain spacing
DEVICE_WIDTH   = 0.6     # Total lateral width
CHANNEL_DEPTH  = 0.2     # Active layer depth (µm)

# Default doping & workfunction
Nd_default      = 1e17   # Channel doping (cm⁻³)
phi_m_default   = 4.87   # Default gate workfunction (eV)

# Derived quantities
phi_B   = phi_m_default - chi_si
Vbi     = phi_B - kT * np.log(Nc / Nd_default)
a_cm    = CHANNEL_DEPTH * 1e-4          # channel depth in cm
V_P     = -q * Nd_default * a_cm**2 / (2 * eps_si)  # pinch-off voltage (V)

print("MESFET Default Parameters")
print("=" * 45)
print(f"Gate workfunction  φ_m  = {phi_m_default} eV")
print(f"Barrier height     φ_B  = {phi_B:.2f} eV")
print(f"Built-in potential V_bi = {Vbi:.3f} V")
print(f"Channel doping     N_d  = {Nd_default:.0e} cm⁻³")
print(f"Channel depth      a    = {CHANNEL_DEPTH} µm")
print(f"Pinch-off voltage  V_P  = {V_P:.3f} V")

---

## 2. Creating and Running a MESFET Simulation

We start with a basic n-channel MESFET at equilibrium to examine its band structure.

In [ ]:
# Create an n-channel MESFET with equilibrium band-diagram logging
sim_eq = create_mesfet(
    # Geometry
    channel_length  = CHANNEL_LENGTH,
    gate_length     = GATE_LENGTH,
    device_width    = DEVICE_WIDTH,
    channel_depth   = CHANNEL_DEPTH,

    # Doping
    channel_doping  = Nd_default,
    device_type     = "n",

    # Gate
    gate_workfunction = phi_m_default,

    # Log the equilibrium band diagram (horizontal cut mid-channel)
    log_bands_eq    = True,

    # No voltage sweep yet — just equilibrium
    vds_sweep       = None,
)

print("MESFET Configuration")
print("=" * 45)
print(f"Channel length  : {CHANNEL_LENGTH} µm")
print(f"Gate length     : {GATE_LENGTH} µm")
print(f"Device width    : {DEVICE_WIDTH} µm")
print(f"Channel depth   : {CHANNEL_DEPTH} µm")
print(f"Channel doping  : {Nd_default:.0e} cm⁻³")
print(f"Gate workfunction: {phi_m_default} eV")
print(f"Electrodes: 1 = Source, 2 = Drain, 3 = Gate")

In [ ]:
# Visualize the MESFET device structure
sim_eq.device_schematic()

In [ ]:
# Run equilibrium simulation
print("Running MESFET equilibrium simulation...")
result_eq = sim_eq.run()

if result_eq.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result_eq.stderr}")


In [ ]:
# Plot equilibrium band diagram (built-in plotly method)
sim_eq.plot_band_diagram(
    title="N-MESFET Equilibrium Band Diagram",
    backend="plotly"
);

---

### 2.1 Equilibrium Device Profiles

Beyond the band diagram, we can extract carrier concentrations, electrostatic potential, and electric field profiles along the channel at equilibrium. These profiles reveal:

- **Carrier concentrations**: The channel has high electron density ($n \approx N_d$) while minority holes are negligible. Under the gate, the Schottky depletion region pushes electrons down.
- **Electrostatic potential**: Shows the built-in potential at the gate Schottky junction.
- **Electric field**: Peaks at the metal-semiconductor interface where the depletion region begins.

In [ ]:
# Equilibrium simulation with carrier, potential, and electric field logging
from nanohubpadre import Solve

sim_prof = create_mesfet(
    channel_length=CHANNEL_LENGTH, gate_length=GATE_LENGTH,
    device_width=DEVICE_WIDTH, channel_depth=CHANNEL_DEPTH,
    channel_doping=Nd_default, device_type="n",
    gate_workfunction=phi_m_default,
    log_bands_eq=True,
)

# Line cut through the channel midpoint (horizontal cut at mid-channel depth)
total_depth = 0.8 + CHANNEL_DEPTH  # substrate + channel
y_ch = total_depth - CHANNEL_DEPTH / 2

# Add carrier concentration, potential, and E-field logging at equilibrium
sim_prof.log_carriers("eq", x_start=0, x_end=DEVICE_WIDTH, y_start=y_ch, y_end=y_ch)
sim_prof.log_potential("pot_eq", x_start=0, x_end=DEVICE_WIDTH, y_start=y_ch, y_end=y_ch)
sim_prof.log_efield("ef_eq", x_start=0, x_end=DEVICE_WIDTH, y_start=y_ch, y_end=y_ch)

print(f"Line cut at y = {y_ch:.2f} µm (mid-channel)")
print("Running equilibrium simulation with full physics logging...")
result_prof = sim_prof.run()
if result_prof.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result_prof.stderr}")


In [ ]:
# Plot equilibrium profiles: carriers, potential, electric field
try:
    n_eq = sim_prof.outputs.get("neq")
    p_eq = sim_prof.outputs.get("peq")
    pot  = sim_prof.outputs.get("pot_eq")
    ef   = sim_prof.outputs.get("ef_eq")

    fig_prof = make_subplots(
        rows=1, cols=3,
        subplot_titles=("Carrier Concentrations", "Electrostatic Potential", "Electric Field"),
    )

    # ── Left: carriers (log scale) ──
    fig_prof.add_trace(go.Scatter(
        x=n_eq.x, y=n_eq.y, mode="lines",
        line=dict(color="blue", width=2), name="Electrons (n)",
    ), row=1, col=1)
    fig_prof.add_trace(go.Scatter(
        x=p_eq.x, y=p_eq.y, mode="lines",
        line=dict(color="red", width=2), name="Holes (p)",
    ), row=1, col=1)

    # ── Center: potential ──
    fig_prof.add_trace(go.Scatter(
        x=pot.x, y=pot.y, mode="lines",
        line=dict(color="green", width=2), name="Potential",
    ), row=1, col=2)

    # ── Right: E-field ──
    fig_prof.add_trace(go.Scatter(
        x=ef.x, y=ef.y, mode="lines",
        line=dict(color="darkorange", width=2), name="E-field",
    ), row=1, col=3)

    fig_prof.update_yaxes(type="log", title_text="Concentration (cm⁻³)", row=1, col=1)
    fig_prof.update_yaxes(title_text="Potential (V)", row=1, col=2)
    fig_prof.update_yaxes(title_text="Electric Field (V/cm)", row=1, col=3)
    fig_prof.update_xaxes(title_text="Position (µm)")
    fig_prof.update_layout(
        title_text="Equilibrium Device Profiles (horizontal cut, mid-channel)",
        template="plotly_white", width=1100, height=430,
    )
    fig_prof.show()

except Exception as e:
    print(f"Could not plot profiles: {e}")
    print("Run the equilibrium simulation cell first.")

---

## 3. Output Characteristic (Id vs Vds)

The **output characteristic** plots drain current $I_d$ against drain-source voltage $V_{ds}$ at a fixed gate-source voltage $V_{gs}$. Two operating regions are visible:

| Region | Condition | Behaviour |
|---|---|---|
| **Linear** | $V_{ds} < V_{ds,\text{sat}}$ | $I_d$ rises roughly linearly with $V_{ds}$ |
| **Saturation** | $V_{ds} \geq V_{ds,\text{sat}}$ | Channel pinches off at the drain; $I_d$ levels off |

We run a single output-characteristic sweep at $V_{gs} = 0\,\text{V}$ (fully open channel).

In [ ]:
# Output characteristic at Vgs = 0 V  (channel fully open)
sim_oc = create_mesfet(
    channel_length    = CHANNEL_LENGTH,
    gate_length       = GATE_LENGTH,
    device_width      = DEVICE_WIDTH,
    channel_depth     = CHANNEL_DEPTH,
    channel_doping    = Nd_default,
    device_type       = "n",
    gate_workfunction = phi_m_default,

    log_iv   = True,
    iv_file  = "idvd_vgs0",

    vgs      = 0.0,                     # Gate grounded
    vds_sweep = (0.0, 2.0, 0.05),       # Sweep Vds: 0 → 2 V
)

print("Running output characteristic (Vgs = 0 V)...")
result_oc = sim_oc.run()
if result_oc.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result_oc.stderr}")


In [ ]:
# Plot the output characteristic
iv_oc = sim_oc.get_iv_data()
Vds_oc, Id_oc = iv_oc.get_iv_data(electrode=2)   # electrode 2 = drain
Vds_oc = np.array(Vds_oc)
Id_oc  = np.abs(np.array(Id_oc))

fig_oc = go.Figure()
fig_oc.add_trace(go.Scatter(
    x=Vds_oc,
    y=Id_oc * 1e6,          # µA
    mode="lines",
    line=dict(color="royalblue", width=2.5),
    name="Vgs = 0 V",
))

fig_oc.update_layout(
    title="N-MESFET Output Characteristic (Vgs = 0 V)",
    xaxis_title="Drain Voltage Vds (V)",
    yaxis_title="Drain Current Id (µA)",
    template="plotly_white",
    width=700, height=450,
)
fig_oc.show()

---

## 4. Transfer Characteristic (Id vs Vgs)

The **transfer characteristic** shows how $I_d$ depends on gate voltage $V_{gs}$ at a fixed (small) $V_{ds}$. It reveals:

- The **threshold voltage** $V_{th}$: gate voltage below which the channel is fully depleted and current drops to near zero.
- The **transconductance** $g_m = \partial I_d / \partial V_{gs}$: a key figure of merit for amplifier gain.

For a depletion-mode device like the n-MESFET the channel conducts at $V_{gs} = 0$ and is pinched off at a negative $V_{gs}$.

In [ ]:
# Transfer characteristic: sweep Vgs at small fixed Vds
# We build the simulation manually to control the Vgs sweep direction

from nanohubpadre import Solve, Log

sim_tc = create_mesfet(
    channel_length    = CHANNEL_LENGTH,
    gate_length       = GATE_LENGTH,
    device_width      = DEVICE_WIDTH,
    channel_depth     = CHANNEL_DEPTH,
    channel_doping    = Nd_default,
    device_type       = "n",
    gate_workfunction = phi_m_default,
    log_iv            = True,
    iv_file           = "idvg",
    # No built-in sweep — we add solves manually below
    vds_sweep         = None,
)

# 1. Equilibrium
sim_tc.add_solve(Solve(initial=True, outfile="eq"))

# 2. Raise Vds to 0.1 V (small, keeps device in linear region for transfer curve)
sim_tc.add_solve(Solve(project=True, v2=0.0, vstep=0.1, nsteps=1,
                        electrode=2, outfile="vds_set"))

# 3. Sweep Vgs from 0 V down to -1.5 V (20 steps of -0.075 V)
sim_tc.add_solve(Solve(project=True, v3=0.0, vstep=-0.075, nsteps=20,
                        electrode=3, outfile="idvg"))

print("Running transfer characteristic (Vds = 0.1 V, Vgs: 0 → −1.5 V)...")
result_tc = sim_tc.run()
if result_tc.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result_tc.stderr}")


In [ ]:
# Plot transfer characteristic — linear and log panels
iv_tc  = sim_tc.get_iv_data()

print(f"IV data: {len(iv_tc.bias_points)} bias points, {iv_tc.num_electrodes} electrodes")

# Use the dedicated transfer-characteristic extractor:
# x = V_gate, y = |I_drain|
Vgs_tc, Id_drain = iv_tc.get_transfer_characteristic(gate_electrode=3, drain_electrode=2)
Vgs_tc  = np.array(Vgs_tc)
Id_drain = np.array(Id_drain)   # already absolute

print(f"Vgs range : {Vgs_tc.min():.3f} … {Vgs_tc.max():.3f} V  ({len(Vgs_tc)} points)")
print(f"Id  range : {Id_drain.min():.3e} … {Id_drain.max():.3e} A")

fig_tc = make_subplots(rows=1, cols=2, subplot_titles=(
    "Linear Scale", "Log Scale"
))

# ── linear panel ──
fig_tc.add_trace(go.Scatter(
    x=Vgs_tc, y=Id_drain * 1e6,
    mode="lines", line=dict(color="royalblue", width=2.5),
    name="Vds = 0.1 V",
), row=1, col=1)

# ── log panel ──
mask = Id_drain > 0
fig_tc.add_trace(go.Scatter(
    x=Vgs_tc[mask], y=Id_drain[mask],
    mode="lines", line=dict(color="royalblue", width=2.5),
    name="Vds = 0.1 V", showlegend=False,
), row=1, col=2)

fig_tc.update_layout(
    title_text="N-MESFET Transfer Characteristic",
    template="plotly_white",
    width=1000, height=450,
)
fig_tc.update_xaxes(title_text="Gate Voltage Vgs (V)")
fig_tc.update_yaxes(title_text="Drain Current Id (µA)", row=1, col=1)
fig_tc.update_yaxes(title_text="Drain Current Id (A)",  type="log", row=1, col=2)

fig_tc.show()

In [ ]:
# Extract threshold voltage and transconductance from the transfer data
# Threshold: constant-current method at Id = 1 nA
I_thresh = 1e-9   # 1 nA

# Sort by Vgs ascending for interpolation
order    = np.argsort(Vgs_tc)
Vgs_sort = Vgs_tc[order]
Id_sort  = Id_drain[order]

# Find Vth: first Vgs where Id crosses I_thresh (scanning from negative to 0)
idx_cross = np.where(Id_sort > I_thresh)[0]
if len(idx_cross) > 0:
    Vth_sim = Vgs_sort[idx_cross[0]]
else:
    Vth_sim = np.nan

# Transconductance gm = dId / dVgs  (numerical derivative)
gm = np.gradient(Id_sort, Vgs_sort)
gm_max     = np.max(gm)
gm_max_idx = np.argmax(gm)
Vgs_at_gm  = Vgs_sort[gm_max_idx]

# Compare with analytical estimates
Vth_theory = V_P - Vbi   # approximate pinch-off condition

print("Extracted MESFET Parameters")
print("=" * 45)
print(f"Threshold voltage  (sim, 1 nA method) : {Vth_sim:.3f} V")
print(f"Threshold voltage  (theory V_P − V_bi): {Vth_theory:.3f} V")
print(f"Peak transconductance gm_max          : {gm_max*1e6:.2f} µA/V")
print(f"  at Vgs = {Vgs_at_gm:.3f} V")

---

## 5. Output Characteristic Family (Multiple Vgs)

Running the output characteristic at several gate voltages gives the classic **Id–Vds family** used to characterise FETs. Each curve saturates at a lower current as $V_{gs}$ becomes more negative.

In [ ]:
# Sweep Vds at several Vgs values
VGS_POINTS = [0.0, -0.2, -0.4, -0.6, -0.8]   # V
COLORS      = ["royalblue", "darkorange", "green", "red", "purple"]

oc_data = {}   # Vgs → (Vds_array, Id_array)

for vgs_val in VGS_POINTS:
    tag = f"vgs{vgs_val:.1f}".replace("-", "m").replace(".", "p")
    sim = create_mesfet(
        channel_length    = CHANNEL_LENGTH,
        gate_length       = GATE_LENGTH,
        device_width      = DEVICE_WIDTH,
        channel_depth     = CHANNEL_DEPTH,
        channel_doping    = Nd_default,
        device_type       = "n",
        gate_workfunction = phi_m_default,
        log_iv   = True,
        iv_file  = f"idvd_{tag}",
        vgs      = vgs_val,
        vds_sweep = (0.0, 2.0, 0.05),
    )
    print(f"  Running Vgs = {vgs_val:+.1f} V …", end=" ")
    result = sim.run()
    if result.returncode != 0:
        raise RuntimeError(f"Simulation failed:\n{result.stderr}")
    iv   = sim.get_iv_data()
    Vds, Id = iv.get_iv_data(electrode=2)
    oc_data[vgs_val] = (np.array(Vds), np.abs(np.array(Id)))
    print("done")


In [ ]:
# Plot the output-characteristic family
fig_oc_fam = go.Figure()

for i, vgs_val in enumerate(VGS_POINTS):
    if vgs_val not in oc_data:
        continue
    Vds, Id = oc_data[vgs_val]
    fig_oc_fam.add_trace(go.Scatter(
        x=Vds, y=Id * 1e6,
        mode="lines",
        line=dict(color=COLORS[i], width=2.2),
        name=f"Vgs = {vgs_val:+.1f} V",
    ))

fig_oc_fam.update_layout(
    title="N-MESFET Output Characteristics",
    xaxis_title="Drain Voltage Vds (V)",
    yaxis_title="Drain Current Id (µA)",
    template="plotly_white",
    width=750, height=480,
)
fig_oc_fam.show()

---

## 5.1 2D Potential Energy Maps

A heat map of the electrostatic potential across the device cross-section reveals the spatial distribution of the built-in field, the gate depletion region, and how the potential landscape changes under drain bias. Contour lines highlight the equipotential surfaces.

PADRE's `PLOT.3D` command dumps mesh-node data (x, y, z, value) as a scatter file. We interpolate that scattered data onto a regular grid using `scipy.interpolate.griddata` (the same approach used by the Rappture MESFET Lab), then overlay Plotly heat maps with contour lines.

We compare two conditions:
- **Equilibrium** ($V_{ds} = 0$): the potential is set by doping profiles and Schottky gate alone.
- **Final bias** ($V_{ds} = 2\,\text{V}$): the drain-side potential drops, pinching the channel.

In [ ]:
# Run a simulation that saves Plot3D scatter files at equilibrium and final bias
# The contour_maps=True parameter automatically adds PLOT.3D commands to the deck

sim_2d = create_mesfet(
    channel_length=CHANNEL_LENGTH, gate_length=GATE_LENGTH,
    device_width=DEVICE_WIDTH, channel_depth=CHANNEL_DEPTH,
    channel_doping=Nd_default, device_type="n",
    gate_workfunction=phi_m_default,
    # Enable 2D contour maps: dumps Plot3D scatter files at equilibrium and Vds bias
    contour_maps=True,
    contour_vds_bias=2.0,    # Drain-source voltage for bias contour maps
    contour_quantities=["potential", "doping"],  # Only pot and doping for this section
)

print("Running simulation for 2D maps...")
print("  Equilibrium → pot_eq, dop_eq (Plot3D)")
print("  Final bias (Vds = 2.0V) → pot_bias (Plot3D)")
result_2d = sim_2d.run()
if result_2d.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result_2d.stderr}")


In [ ]:
# ── 1. Potential Energy Maps: Equilibrium vs Final Bias ──
sim_2d.plot_contour(["pot_eq", "pot_bias"],
                    title=["Potential — Equilibrium (Vds = 0 V)",
                           "Potential — Vds = 2.0 V"],
                    cbar_title="V");

# ── 2. Doping Profile Map ──
sim_2d.plot_contour("dop_eq",
                    title="2D Doping Profile (log scale)",
                    colorscale="Jet", log_scale=True,
                    cbar_title="Doping (cm⁻³)");

---

## 5.2 Band Diagram & Carrier Evolution Under Bias

As the drain-source voltage $V_{ds}$ increases, the energy bands on the drain side shift downward. The quasi-Fermi level for electrons ($E_{fn}$) splits from the equilibrium Fermi level, reflecting the current flow through the channel. Simultaneously, the electron concentration near the drain decreases as the depletion region widens.

Comparing these profiles at several $V_{ds}$ values reveals the transition from linear to saturation operation.

In [ ]:
# Build a custom simulation that logs band diagrams and carriers at each Vds step
from nanohubpadre import Solve

VDS_STEPS = [0.0, 0.5, 1.0, 1.5, 2.0]  # Bias points for profile snapshots

sim_bias = create_mesfet(
    channel_length=CHANNEL_LENGTH, gate_length=GATE_LENGTH,
    device_width=DEVICE_WIDTH, channel_depth=CHANNEL_DEPTH,
    channel_doping=Nd_default, device_type="n",
    gate_workfunction=phi_m_default,
)

total_depth = 0.8 + CHANNEL_DEPTH
y_ch = total_depth - CHANNEL_DEPTH / 2

# Equilibrium solve + logging
sim_bias.add_solve(Solve(initial=True, outfile="eq"))
sim_bias.log_band_diagram("eq", x_start=0, x_end=DEVICE_WIDTH, y_start=y_ch, y_end=y_ch, include_qf=True)
sim_bias.log_carriers("eq", x_start=0, x_end=DEVICE_WIDTH, y_start=y_ch, y_end=y_ch)

# Step through each Vds and log bands + carriers
for i, vds in enumerate(VDS_STEPS):
    if i == 0:
        continue  # equilibrium already logged
    tag = f"vds{int(vds*10)}"
    # Step from previous voltage to this one
    v_prev = VDS_STEPS[i - 1]
    step_size = vds - v_prev
    sim_bias.add_solve(Solve(
        project=True, v2=v_prev, vstep=step_size, nsteps=1,
        electrode=2, outfile=tag
    ))
    sim_bias.log_band_diagram(tag, x_start=0, x_end=DEVICE_WIDTH,
                               y_start=y_ch, y_end=y_ch, include_qf=True)
    sim_bias.log_carriers(tag, x_start=0, x_end=DEVICE_WIDTH,
                           y_start=y_ch, y_end=y_ch)

print(f"Logging band diagrams and carriers at Vds = {VDS_STEPS}")
print("Running simulation...")
result_bias = sim_bias.run()
if result_bias.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result_bias.stderr}")


In [ ]:
# Plot band diagram evolution: Ec, Ev, Efn at each Vds
fig_bands = go.Figure()

band_colors = ['royalblue', 'darkorange', 'green', 'red', 'purple']
VDS_TAGS  = ['eq'] + [f'vds{int(v*10)}' for v in VDS_STEPS[1:]]

for i, (vds, tag) in enumerate(zip(VDS_STEPS, VDS_TAGS)):
    color = band_colors[i]
    try:
        ec  = sim_bias.outputs.get(f"cb{tag}")
        ev  = sim_bias.outputs.get(f"vb{tag}")
        efn = sim_bias.outputs.get(f"qfn{tag}")

        # Ec (solid)
        fig_bands.add_trace(go.Scatter(
            x=ec.x, y=ec.y, mode="lines",
            line=dict(color=color, width=2),
            name=f"Vds={vds:.1f}V",
            legendgroup=f"vds{i}",
        ))
        # Ev (solid, thinner)
        fig_bands.add_trace(go.Scatter(
            x=ev.x, y=ev.y, mode="lines",
            line=dict(color=color, width=1.5),
            showlegend=False, legendgroup=f"vds{i}",
        ))
        # Efn (dashed)
        fig_bands.add_trace(go.Scatter(
            x=efn.x, y=efn.y, mode="lines",
            line=dict(color=color, width=1.5, dash="dash"),
            showlegend=False, legendgroup=f"vds{i}",
        ))
    except Exception as e:
        print(f"Could not load band data for Vds={vds}V: {e}")

fig_bands.update_layout(
    title="Band Diagram Evolution with Drain Bias (Ec, Ev solid; Efn dashed)",
    xaxis_title="Position (µm)",
    yaxis_title="Energy (eV)",
    template="plotly_white",
    width=900, height=500,
)
fig_bands.show()

In [ ]:
# Plot carrier concentration evolution at each Vds
fig_carriers = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Electron Concentration vs Position", "Hole Concentration vs Position"),
)

for i, (vds, tag) in enumerate(zip(VDS_STEPS, VDS_TAGS)):
    color = band_colors[i]
    try:
        n_data = sim_bias.outputs.get(f"n{tag}")
        p_data = sim_bias.outputs.get(f"p{tag}")

        fig_carriers.add_trace(go.Scatter(
            x=n_data.x, y=n_data.y, mode="lines",
            line=dict(color=color, width=2),
            name=f"Vds={vds:.1f}V",
        ), row=1, col=1)

        fig_carriers.add_trace(go.Scatter(
            x=p_data.x, y=p_data.y, mode="lines",
            line=dict(color=color, width=2),
            showlegend=False, name=f"Vds={vds:.1f}V",
        ), row=1, col=2)
    except Exception as e:
        print(f"Could not load carrier data for Vds={vds}V: {e}")

fig_carriers.update_yaxes(type="log", title_text="n (cm⁻³)", row=1, col=1)
fig_carriers.update_yaxes(type="log", title_text="p (cm⁻³)", row=1, col=2)
fig_carriers.update_xaxes(title_text="Position (µm)")
fig_carriers.update_layout(
    title_text="Carrier Concentration Evolution with Drain Bias",
    template="plotly_white", width=1000, height=450,
)
fig_carriers.show()

print("\nKey observations:")
print("- Electrons near drain decrease as Vds increases (channel depletion)")
print("- At high Vds the channel pinches off near the drain → saturation")
print("- Hole concentration remains very low throughout (n-channel device)")

---

## 6. Effect of Gate Workfunction

The gate metal workfunction $\phi_m$ determines the Schottky barrier height $\phi_B = \phi_m - \chi$. A higher $\phi_m$ creates a taller barrier, increasing $V_{bi}$ and making the channel easier to deplete — effectively shifting the threshold voltage more negative. We compare three representative metals.

In [ ]:
# Gate-workfunction sweep
METALS = {
    "Ti  (φ_m = 4.33 eV)": 4.33,
    "W   (φ_m = 4.55 eV)": 4.55,
    "Pt  (φ_m = 5.10 eV)": 5.10,
}
METAL_COLORS = ["#1f77b4", "#ff7f0e", "#2ca02c"]

wf_data = {}   # label → (Vds, Id)

for (label, phi_m), color in zip(METALS.items(), METAL_COLORS):
    print(f"  φ_m = {phi_m} eV ({label.split('(')[0].strip()}) …", end=" ")
    sim = create_mesfet(
        channel_length    = CHANNEL_LENGTH,
        gate_length       = GATE_LENGTH,
        device_width      = DEVICE_WIDTH,
        channel_depth     = CHANNEL_DEPTH,
        channel_doping    = Nd_default,
        device_type       = "n",
        gate_workfunction = phi_m,
        log_iv   = True,
        iv_file  = f"idvd_phi{phi_m}",
        vgs      = 0.0,
        vds_sweep = (0.0, 2.0, 0.05),
    )
    result = sim.run()
    if result.returncode != 0:
        raise RuntimeError(f"Simulation failed:\n{result.stderr}")
    iv   = sim.get_iv_data()
    Vds, Id = iv.get_iv_data(electrode=2)
    wf_data[label] = (np.array(Vds), np.abs(np.array(Id)))
    print("done")


In [ ]:
# Plot workfunction comparison — output characteristic (left) + theory Vth (right)
fig_wf = make_subplots(rows=1, cols=2, subplot_titles=(
    "Output Characteristic at Vgs = 0 V",
    "Built-in Potential & Pinch-off Voltage vs φ_m"
))

# ── left: Id vs Vds ──
for (label, _), color in zip(METALS.items(), METAL_COLORS):
    if label not in wf_data:
        continue
    Vds, Id = wf_data[label]
    fig_wf.add_trace(go.Scatter(
        x=Vds, y=Id * 1e6,
        mode="lines", line=dict(color=color, width=2.2),
        name=label,
    ), row=1, col=1)

# ── right: theoretical Vbi and |V_P| vs workfunction ──
phi_ms  = np.array(list(METALS.values()))
phi_Bs  = phi_ms - chi_si
Vbis    = phi_Bs - kT * np.log(Nc / Nd_default)
V_Ps    = -q * Nd_default * a_cm**2 / (2 * eps_si)   # scalar — same for all
Vths    = V_Ps - Vbis                                  # threshold shifts with Vbi

fig_wf.add_trace(go.Scatter(
    x=phi_ms, y=Vbis,
    mode="lines+markers", line=dict(color="#636EFA", width=2),
    marker=dict(size=8), name="V_bi",
), row=1, col=2)

fig_wf.add_trace(go.Scatter(
    x=phi_ms, y=Vths,
    mode="lines+markers", line=dict(color="#EF553B", width=2),
    marker=dict(size=8), name="V_th (theory)",
), row=1, col=2)

fig_wf.update_layout(
    title_text="Effect of Gate Workfunction",
    template="plotly_white",
    width=1050, height=470,
)
fig_wf.update_xaxes(title_text="Drain Voltage Vds (V)",      row=1, col=1)
fig_wf.update_yaxes(title_text="Drain Current Id (µA)",      row=1, col=1)
fig_wf.update_xaxes(title_text="Gate Workfunction φ_m (eV)", row=1, col=2)
fig_wf.update_yaxes(title_text="Voltage (V)",                row=1, col=2)

fig_wf.show()

---

## 7. Effect of Channel Doping

Channel doping $N_d$ controls both the carrier concentration (and hence channel resistance) and the depletion-width scaling. Higher doping yields more current but also requires a larger gate swing to pinch off. We sweep $N_d$ while keeping geometry and workfunction fixed.

In [ ]:
# Channel-doping sweep — output characteristic at Vgs = 0
DOPINGS       = [5e16, 1e17, 5e17]
DOPING_COLORS = ["#9467bd", "#d62728", "#8c564b"]
DOPING_LABELS = [f"{n:.0e} cm⁻³" for n in DOPINGS]

dop_data = {}   # Nd → (Vds, Id)

for Nd, label in zip(DOPINGS, DOPING_LABELS):
    print(f"  Nd = {label} …", end=" ")
    sim = create_mesfet(
        channel_length    = CHANNEL_LENGTH,
        gate_length       = GATE_LENGTH,
        device_width      = DEVICE_WIDTH,
        channel_depth     = CHANNEL_DEPTH,
        channel_doping    = Nd,
        device_type       = "n",
        gate_workfunction = phi_m_default,
        log_iv   = True,
        iv_file  = f"idvd_nd{Nd:.0e}".replace("+", ""),
        vgs      = 0.0,
        vds_sweep = (0.0, 2.0, 0.05),
    )
    result = sim.run()
    if result.returncode != 0:
        raise RuntimeError(f"Simulation failed:\n{result.stderr}")
    iv   = sim.get_iv_data()
    Vds, Id = iv.get_iv_data(electrode=2)
    dop_data[Nd] = (np.array(Vds), np.abs(np.array(Id)))
    print("done")


In [ ]:
# Plot doping comparison — output characteristic (left) + pinch-off voltage (right)
fig_dop = make_subplots(rows=1, cols=2, subplot_titles=(
    "Output Characteristic at Vgs = 0 V",
    "Pinch-off Voltage vs Channel Doping"
))

# ── left ──
for (Nd, label), color in zip(zip(DOPINGS, DOPING_LABELS), DOPING_COLORS):
    if Nd not in dop_data:
        continue
    Vds, Id = dop_data[Nd]
    fig_dop.add_trace(go.Scatter(
        x=Vds, y=Id * 1e6,
        mode="lines", line=dict(color=color, width=2.2),
        name=f"Nd = {label}",
    ), row=1, col=1)

# ── right: theoretical |V_P| vs Nd ──
Nd_range = np.logspace(16, 18, 60)
VP_range = q * Nd_range * a_cm**2 / (2 * eps_si)  # |V_P|

fig_dop.add_trace(go.Scatter(
    x=Nd_range, y=-VP_range,
    mode="lines", line=dict(color="#17becf", width=2.5),
    name="|V_P| (theory)", showlegend=True,
), row=1, col=2)

# Mark the three simulated dopings
for Nd, color in zip(DOPINGS, DOPING_COLORS):
    vp_mark = -q * Nd * a_cm**2 / (2 * eps_si)
    fig_dop.add_trace(go.Scatter(
        x=[Nd], y=[vp_mark],
        mode="markers", marker=dict(size=12, color=color, line=dict(color="black", width=1)),
        name=f"Nd = {Nd:.0e}",
        showlegend=False,
    ), row=1, col=2)

fig_dop.update_layout(
    title_text="Effect of Channel Doping",
    template="plotly_white",
    width=1050, height=470,
)
fig_dop.update_xaxes(title_text="Drain Voltage Vds (V)",   row=1, col=1)
fig_dop.update_yaxes(title_text="Drain Current Id (µA)",   row=1, col=1)
fig_dop.update_xaxes(title_text="Channel Doping Nd (cm⁻³)", type="log", row=1, col=2)
fig_dop.update_yaxes(title_text="Pinch-off Voltage V_P (V)", row=1, col=2)

fig_dop.show()

---

## 8. Exercises

### Exercise 1: Deeper Channel
Increase `channel_depth` to 0.4 µm (keeping doping at 1e17 cm⁻³). How does the saturation current change? Does $V_P$ shift, and in which direction?

### Exercise 2: P-channel MESFET
Set `device_type="p"` and choose an appropriate workfunction. Run an output characteristic and compare the current polarities with the n-channel case.

### Exercise 3: High-Field Effects
Run two output sweeps at $V_{gs} = 0$ V with `fldmob=True` (default) and `fldmob=False`. Compare the saturation regions — field-dependent mobility limits current at high lateral fields.

In [ ]:
# Exercise 1 skeleton — deeper channel
sim_ex1 = create_mesfet(
    channel_depth     = 0.4,          # ← change this
    channel_doping    = 1e17,
    device_type       = "n",
    gate_workfunction = phi_m_default,
    log_iv            = True,
    iv_file           = "ex1_deep",
    vgs               = 0.0,
    vds_sweep         = (0.0, 2.0, 0.05),
)

# Theoretical pinch-off voltage for a = 0.4 µm
a_ex1  = 0.4e-4   # cm
VP_ex1 = -q * Nd_default * a_ex1**2 / (2 * eps_si)
print(f"Exercise 1 — deeper channel (a = 0.4 µm)")
print(f"  Theoretical V_P = {VP_ex1:.3f} V  (compare with default {V_P:.3f} V)")
print(f"  Running simulation …")

result_ex1 = sim_ex1.run()
if result_ex1.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result_ex1.stderr}")
iv_ex1       = sim_ex1.get_iv_data()
Vds_ex1, Id_ex1 = iv_ex1.get_iv_data(electrode=2)
print("  Done!")

    fig_ex1 = go.Figure()
    fig_ex1.add_trace(go.Scatter(
        x=Vds_ex1, y=np.abs(np.array(Id_ex1)) * 1e6,
        mode="lines", line=dict(color="royalblue", width=2.5),
        name="a = 0.4 µm",
    ))
    # overlay default for comparison
    if 0.0 in oc_data:
        Vds_def, Id_def = oc_data[0.0]
        fig_ex1.add_trace(go.Scatter(
            x=Vds_def, y=Id_def * 1e6,
            mode="lines", line=dict(color="gray", width=2, dash="dash"),
            name="a = 0.2 µm (default)",
        ))

    fig_ex1.update_layout(
        title="Exercise 1 — Deeper Channel Comparison",
        xaxis_title="Drain Voltage Vds (V)",
        yaxis_title="Drain Current Id (µA)",
        template="plotly_white",
        width=700, height=440,
    )
    fig_ex1.show()
else:
    print("  Simulation failed!")

---

## Summary

In this notebook you learned:

1. **MESFET Structure** — Schottky gate replaces the gate oxide; depletion under the gate modulates channel current.
2. **Equilibrium Profiles** — Carrier concentrations, potential, and electric field across the device at zero bias.
3. **Output Characteristics** — Linear and saturation regions visible in Id vs Vds curves.
4. **Transfer Characteristics** — Threshold voltage extraction and transconductance measurement from Id vs Vgs.
5. **Output-Characteristic Family** — Id–Vds curves at multiple Vgs values show the depletion-mode behaviour.
6. **Band & Carrier Evolution** — How energy bands and carrier concentrations change as drain bias increases through linear into saturation.
7. **Gate Workfunction Effect** — Higher $\phi_m$ increases barrier height, shifts threshold more negative, and reduces saturation current at $V_{gs}=0$.
8. **Channel Doping Effect** — Higher $N_d$ raises saturation current but also increases the pinch-off voltage magnitude.

**Key Equations:**
- Barrier height: $\phi_B = \phi_m - \chi$
- Pinch-off voltage: $V_P = -q N_d a^2 / 2\varepsilon_s$
- Threshold: $V_{th} \approx V_P - V_{bi}$

**Next**: [09 - Solar Cell](06_Solar_Cell.ipynb) — Photovoltaic device simulation